In [1]:
from socket import *
from threading import *
import os,stat
import trainLib as lib

allgrids = []
allgridslock = Lock()

gridlocks = [] # dict: key: gridname value: lock
gridlockslock = Lock()

class GridWrapper():
    def __init__(self, grid, gridname):
        self.grid = grid
        self.gridname = gridname
        self.userlist = []
        self.isShared = False

    def createlock(self):
        global gridlocks
        with gridlockslock:
            gridlocks[self.gridname] = Lock()

def worker(sock : socket):
    # serves client with username user1
    # each worker should now the username of the connected client. username will be used in lib calls
    username = "user1"

    # later, this is modified as the result of attach gridname command
    attachedGridName = "default"
    while True:
        error = 'ERROR'.encode()
        space = ' '.encode()
        msg = sock.recv(4096)
        # parse msg
        #m = msg.decode()
        #m = m + " received."
        args = msg.split()
        command = args[0]
        command = command.decode()

        if command == 'CREATE': # CREATE row col
            msg = 'GRID CREATED'.encode()
            row = int(args[1].decode())
            col = int(args[2].decode())
            #func call
            sock.send(args[1]+ 'x'.encode()+args[2]+space+msg)

        elif command == 'ADD': #ADD type row col
            type = args[1].decode()
            row = int(args[2].decode())
            col = int(args[3].decode())
            msg = 'TYPE ELEMENT ADDED AT'.encode()
            #func call
            
            sock.send(args[1] + space +msg + space + args[2] + space + args[3])

        elif command == 'REMOVE': #remove row col
            row = int(args[1].decode())
            col = int(args[2].decode())
            msg = 'ELEMENT REMOVED AT'.encode()
            sock.send(msg + space + args[1] + space + args[2])

        elif command == 'CHSTATE': #chstate row col
            row = int(args[1].decode())
            col = int(args[2].decode())
            msg = 'STATE OF THE SWITCH AT'.encode()
            msg2 = 'CHANGED'.encode
            sock.send(msg + space + args[1] + space + args[2] + space + msg2)
        
        elif command == 'ROTATE':  #rotate times row col
            times = int(args[1].decode())
            row = int(args[2].decode())
            col = int(args[3].decode())
            msg = 'ELEMENT AT'.encode()
            msg2 = 'ROTATED'.encode
            msg3 = 'TIMES'.encode()
            sock.send(msg + space + args[2] + space + args[3] + space + msg2 + space + args[1] + msg3)

        elif command == 'GETDUR':  #getdur row col dir
            row = int(args[1].decode())
            col = int(args[2].decode())
            dir = args[3].decode()
            msg = 'DURATION FOR CELL '.encode()
            sock.send(msg + space + args[1] + space + args[2] +':'.encode()) ##add result

        elif command == 'GETNEXT':  #getnext row col dir
            row = int(args[1].decode())
            col = int(args[2].decode())
            dir = args[3].decode()
            msg = 'NEXTCELL FOR CELL '.encode()
            sock.send(msg + space + args[1] + space + args[2] +':'.encode()) ##add result
        
        elif command == 'GETSTATUS':  #getstatus row col
            row = int(args[1].decode())
            col = int(args[2].decode()) 
            msg = 'STATUS OF CELL '.encode()
            sock.send(msg + space + args[1] + space + args[2] +':'.encode()) ##add result

        elif command == 'GETSTOP':  #getstop row col dir
            row = int(args[1].decode())
            col = int(args[2].decode())
            dir = args[3].decode()
            msg = 'STOP FOR CELL '.encode()
            sock.send(msg + space + args[1] + space + args[2] +':'.encode()) ##add result 
        
        elif command == 'START':  #startsim
            msg='SIMULATION STARTED'.encode()
            sock.send(msg)

        elif command == 'STOP':
            msg='SIMULATION STOPPED'.encode()
            sock.send(msg)

        elif command == 'PAUSE':
            msg='SIMULATION PAUSED'.encode()
            sock.send(msg)

        elif command == 'STOPDSP': 
            msg='DISPLAYING STOPPED'.encode()
            sock.send(msg)

        elif command == 'BYE':
            msg = 'BYE TO YOU'.encode()
            sock.send(msg)

        else:
            sock.send(error)

        ''' 
        # assume command is creategrid gridname row col
        row = 4
        col = 6
        gridname = "default"

        rawgrid = lib.GameGrid(row, col)
        wrappedgrid = GridWrapper(rawgrid, gridname)
        # sync point
        wrappedgrid.createlock()
        wrappedgrid.userlist.append(username)
        wrappedgrid.isShared = False

        with allgridslock:
            allgrids[gridname] = wrappedgrid

        #if command ADD 

        '''

        #sock.send(m.encode())
        #sock.send(command)

def addelm(type, row, col):
    return

def server(ip, port):
    s = socket(AF_INET, SOCK_STREAM)
    s.bind((ip,port))
    s.listen(10)    # 1 is queue size for "not yet accept()'ed connections"
    try:
        while True:
        #for i in range(5):    # just limit # of accepts for Thread to exit
            ns, peer = s.accept()
            print(peer, "connected")
                # create a thread with new socket
            t = Thread(target = worker, args=(ns,))
            t.start()
            # now main thread ready to accept next connection
    finally:
        s.close()


if __name__ == '__main__':
	server('127.0.0.1', 34377)





('127.0.0.1', 52061) connected
